In [ ]:
import json
import re
import sys
import os
import pandas as pd
import ast
# from pandas import json_normalize
from tqdm import tqdm
from collections import Counter
from pathlib import Path
from pandas import json_normalize

In [ ]:
# We'll start by creating a directory in which we'll define our new
# modules to be imported. Also, we import the data that we need for run the code to 
# skip the part of generating KNN and Sparse Graph which is the time consuming part.
# %mkdir data
# %mkdir dataset
# %mkdir embeddings
%mkdir -p Trec_data
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

# %cp -a gdrive/My\ Drive/VolunteerismTransfer/Code/Labeled.json ./
%cp -a gdrive/My\ Drive/VolunteerismTransfer/Code/events_score.csv ./
# %cp -a gdrive/My\ Drive/CorrectAndSmooth/data/. ./data/

Mounted at /content/gdrive


In [ ]:
import sys
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
import nltk
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel

from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt

from gensim.models import KeyedVectors

from nltk.corpus import stopwords
import gensim.downloader as api
import string
from sklearn.semi_supervised import LabelSpreading, LabelPropagation

# **`Clean and Process Data `**

In [ ]:
import nltk
nltk.download()

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(["http", "https", "rt", "@", ":", "t.co", "co", "amp", "&amp;", "...", "\n", "\r"])
stop_words.extend(string.punctuation)

In [ ]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True)
        yield(sent)

In [ ]:

df = pd.read_json("./abeled.json", orient='records',lines=True)
data = df.text.values.tolist()
data_words = list(sent_to_words(data))

In [ ]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]
    return texts_out

In [ ]:
data_ready = process_words(data_words)

In [ ]:
df['processed_text']=data_ready
df.to_json('./PR_all_Labeled.json', orient='records', lines=True)


In [ ]:
cp -a ./PR_all_Labeled.json  gdrive/My\ Drive/Code/

In [ ]:
data = pd.read_json("./PR_all_Labeled.json", orient='records',lines=True)


# Topic Modeling

In [ ]:
id2word = corpora.Dictionary(data_ready)
corpus = [id2word.doc2bow(text) for text in data_ready]


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

pprint(lda_model.print_topics())

[(0,
  '0.036*"collect" + 0.030*"number" + 0.029*"hand" + 0.027*"link" + '
  '0.026*"place" + 0.021*"ready" + 0.021*"provide" + 0.018*"home" + '
  '0.016*"high" + 0.016*"calm"'),
 (1,
  '0.106*"flood" + 0.056*"ymmfire" + 0.046*"due" + 0.031*"fire" + 0.025*"team" '
  '+ 0.025*"update" + 0.018*"late" + 0.017*"evacuation" + 0.013*"find" + '
  '0.012*"instead"'),
 (2,
  '0.149*"need" + 0.038*"train" + 0.025*"send" + 0.023*"official" + '
  '0.019*"say" + 0.017*"good" + 0.016*"information" + 0.016*"give" + '
  '0.015*"supply" + 0.014*"bullet"'),
 (3,
  '0.187*"help" + 0.098*"people" + 0.067*"affect" + 0.031*"thousand" + '
  '0.030*"donate" + 0.023*"go" + 0.018*"follow" + 0.018*"thank" + 0.018*"make" '
  '+ 0.014*"want"'),
 (4,
  '0.064*"open" + 0.034*"shelter" + 0.031*"many" + 0.030*"start" + '
  '0.027*"family" + 0.027*"let" + 0.026*"know" + 0.017*"case" + '
  '0.017*"nepalearthquake" + 0.016*"evacuee"'),
 (5,
  '0.068*"victim" + 0.048*"stay" + 0.041*"use" + 0.036*"resident" + '
  '0.035*"e

In [ ]:
t=lda_model.get_document_topics(corpus,minimum_probability=0.0)

In [ ]:
new_df=pd.DataFrame()
count=0
for index,row in df.iterrows():
    new_df=new_df.append({"postID":row["postID"],"eventid":row["eventid"],"event_type":row["event_type"],"climate_related":row["climate_related"], "t0":t[count][0][1], "t1":t[count][1][1],"t2":t[count][2][1],"t3":t[count][3][1],"t4":t[count][4][1],"t5":t[count][5][1],"t6":t[count][6][1]},ignore_index=True)
    count+=1
    

In [ ]:
new_df.to_csv("./7-topics.csv")

In [ ]:
new_df.groupby('climate_related').mean()

,t0,t1,t2,t3,t4,t5,t6
climate_related,,,,,,,
0.0,0.123584,0.113220,0.161435,0.209775,0.123225,0.126940,0.140575
1.0,0.122463,0.140503,0.160069,0.193850,0.122806,0.123862,0.134532


In [ ]:
new_df.query("climate_related == 1").groupby('event_type').mean()

,climate_related,t0,t1,t2,t3,t4,t5,t6
event_type,,,,,,,,
flood,1.0,0.108854,0.084057,0.199707,0.152625,0.184916,0.161173,0.109790
hurricane/typhoon/cyclone/tornado,1.0,0.110128,0.094879,0.255910,0.112118,0.190297,0.091692,0.144564
wildfire/bushfire,1.0,0.198788,0.114797,0.183758,0.102120,0.153320,0.123913,0.122381


In [ ]:
new_df.query("climate_related == 1").groupby('eventid').mean()

,climate_related,t0,t1,t2,t3,t4,t5,t6
eventid,,,,,,,,
2014_Hurricane_Odile_Mexico,1.0,0.027043,0.027033,0.027038,0.067501,0.595820,0.228524,0.027026
2014_Philippines_Typhoon_Hagupi,1.0,0.119366,0.136584,0.142342,0.135089,0.152557,0.172776,0.142841
2015_Cyclone_Pam,1.0,0.159113,0.141851,0.286166,0.215525,0.095320,0.027702,0.074331
albertaFloods2013,1.0,0.115432,0.107207,0.189555,0.257680,0.097860,0.084687,0.147589
albertaWildfires2019,1.0,0.159760,0.145961,0.116644,0.186656,0.158377,0.117764,0.114829
australiaBushfire2013,1.0,0.197435,0.173929,0.107807,0.179648,0.118767,0.071981,0.150462
cycloneKenneth2019,1.0,0.148231,0.142328,0.190495,0.192108,0.107081,0.101133,0.118618
fireColorado2012,1.0,0.138547,0.112463,0.209323,0.186586,0.148174,0.117755,0.087115
fireYMM2016,1.0,0.077697,0.207160,0.136828,0.169315,0.113475,0.128672,0.166875


# Generating Similarity Scores and Matrix
### **Mean and Cosine Similarity(each event with all other event-types)**


In [ ]:
import pickle
import logging
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.pipeline import make_pipeline
from scipy import sparse
import numpy as np
import os.path
import re
from tqdm import tqdm
import warnings
import pandas as pd

In [ ]:
# remove irrelevant tweets
data = pd.read_json("./PR_all_Labeled.json", orient='records',lines=True)
data.loc[(data.eventid == 'parisAttacks2015'),'event_type']='shooting'
def label_ir_tweets(postCategories):

    if 'Irrelevant' in postCategories:
        return 1
    else:
        return 0
data['ir'] = [label_ir_tweets(x) for x in data['categories']]
data=data.query("ir == 0")
data=data.query("label ==1")

In [ ]:
def label_observation_tweets(postCategories):

    if 'FirstPartyObservation' in postCategories or 'ThirdPartyObservation' in postCategories :
        return 1
    else:
        return 0
data['obs'] = [label_observation_tweets(x) for x in data['categories']]
data=data.query("obs == 1")
# data=data.query("label ==1")

In [ ]:
data['l'] = data.apply(lambda row: len(row['processed_text']), axis=1)
data= data.query("l >1")
data.drop(columns=['l'], inplace=True)

In [ ]:
pip install -U sentence-transformers

In [ ]:
# generate sentnece embedding
class SBERT:

    def __init__(self, lang="en"):
        from sentence_transformers import SentenceTransformer
        self.name = "SBERT"
        if lang == "fr":
            self.model = SentenceTransformer(
                "/home/bmazoyer/Dev/pytorch_bert/output/sts_fr_long_multilingual_bert-2019-10-01_15-07-03")
        elif lang == "en":
            self.model = SentenceTransformer(
                # "bert-large-nli-stsb-mean-tokens"
                "roberta-large-nli-stsb-mean-tokens"
            )
# roberta-large-nli-stsb-mean-tokens
    def compute_vectors(self, data):
        data["text"] = data.text.str.slice(0, 500)
        vectors = np.array(self.model.encode(data.text.tolist()))
        return vectors

In [ ]:
sbert=SBERT()

In [ ]:
v=sbert.compute_vectors(data)
data['sbert_emb']=[item for item in v]

In [ ]:
events=[
'2014_Philippines_Typhoon_Hagupi',
 '2015_Cyclone_Pam',
 'albertaFloods2013',
 'albertaWildfires2019',
 'australiaBushfire2013',
 'cycloneKenneth2019',
 'fireYMM2016',
 'hurricaneFlorence2018',
 'keralaFloods2019',
 'manilaFloods2013',
 'philipinnesFloods2012',
 'queenslandFloods2013',
 'southAfricaFloods2019',
 'typhoonHagupit2014',
 'typhoonYolanda2013'
]
event_types=[ 'hurricane/typhoon/cyclone/tornado','flood','wildfire/bushfire']

In [ ]:
from numpy import dot
from numpy.linalg import norm

In [ ]:
#generate similarity scores dataframe
event_ranks=pd.DataFrame()
groupby_col='event_type'
event_types=data.query("label==1").groupby("event_type")
for heldout_event,g in event_types:

    training = data[data[groupby_col] != heldout_event]
    test = data[data[groupby_col] == heldout_event]

    ref=np.mean(test["sbert_emb"], axis=0)

    grpups=training.query("label==1").groupby("event_type")
    ranks={}
    ranks["reference-event"]=heldout_event
    for name, group in grpups:
       val=np.mean(group["sbert_emb"], axis=0)
       cos_sim = dot(ref, val)/(norm(ref)*norm(val))
       ranks[name]=cos_sim

    # event_ranks[heldout_event]=ranks
    event_ranks = event_ranks.append(ranks, ignore_index=True)
event_ranks.set_index("reference-event",inplace=True)
event_ranks


In [ ]:
event_ranks.to_csv("./event_ranks.csv")
cp -a ./event_ranks_roberta.csv gdrive/My\ Drive/Code/

In [ ]:
# Visualize similarity matrix using heatmap
import seaborn as sns
event_ranks.set_index("reference-event",inplace=True)
p1 = sns.heatmap(event_ranks,cmap="YlGnBu", annot=True)


# Classification Pipeline

In [ ]:
%cp -a gdrive/My\ Drive/VolunteerismTransfer/Code/evaluate.py ./
%cp -a gdrive/My\ Drive/VolunteerismTransfer/Code/PR_all_Labeled.json ./
%cp -a gdrive/My\ Drive/VolunteerismTransfer/Code/evalute_Sampling_Strategies.sh ./

In [ ]:
data=pd.read_json("./PR_all_Labeled.json", orient='records',lines=True)

In [ ]:
!bash evalute_Sampling_Strategies.sh

Start evanulating model for sampling strategy: up-with-similarity-eventCategory with up-weighting=False  and  label spreading=none on albertaFloods2013 with seed number 264152259
finish evanulating model for sampling strategy: up-with-similarity-eventCategory with up-weighting=False  and  label spreading=none on albertaFloods2013 with seed number 264152259
Start evanulating model for sampling strategy: up-with-similarity-eventCategory with up-weighting=False  and  label spreading=none on albertaWildfires2019 with seed number 264152259
finish evanulating model for sampling strategy: up-with-similarity-eventCategory with up-weighting=False  and  label spreading=none on albertaWildfires2019 with seed number 264152259
Start evanulating model for sampling strategy: up-with-similarity-eventCategory with up-weighting=False  and  label spreading=none on australiaBushfire2013 with seed number 264152259
finish evanulating model for sampling strategy: up-with-similarity-eventCategory with up-weig

In [ ]:
cp -a ./result/. gdrive/My\ Drive/VolunteerismTransfer/Code/result_new/